In [33]:
import numpy as np
from AbstractMesh import AbstractMesh
import utils
from scipy.sparse import *

In [2]:
class Trimesh(AbstractMesh):
    
    def __init__(self, vertices= None, faces = None, face_normals = None, labels = None):
        
        self.face_normals         = None #npArray (Nx3)
        self.face_labels       = None #npArray (Nx1)
        self.faces            = None #npArray (Nx3)
        self.face2face         = None #npArray (Nx3?)

        if vertices and faces:
            
            self.vertices = np.array(vertices) 
            self.faces = np.array(faces)
            
            if face_normals:
                self.face_normals = np.array(face_normals)
            
            if labels:
                self.labels = np.array(labels)
        
        super(Trimesh, self).__init__()
     
    
    def show(self):
        pass
    
    def add_face(self,face_id0, face_id1, face_id2):
        new_face = np.array([face_id0, face_id1, face_id2])
        self.faces = np.concatenate([self.faces, new_face])
        self.__compute_adjacencies()
    
    def remove_face(self,face_id):
        self.faces = np.delete(self.faces, face_id)
        self.__compute_adjacencies()
    
    def remove_vertex(self,vtx_id):
        
        self.vertices = np.delete(self.vertices, vtx_id)
        self.faces = self.faces[(self.faces[:,0] != vtx_id) & 
                                (self.faces[:,1] != vtx_id) & 
                                (self.faces[:,2] != vtx_id)] 
        self.__compute_adjacencies()
        
    
    def load_from_file(self, filename):
        
        self.vertices, self.faces, self.face_normals = utils.read_obj(filename)
        self.__compute_adjacencies()
    
    def __compute_adjacencies(self):
        raise NotImplementedError
    
    
    def save_file(self, filename):
        raise NotImplementedError
        
    @property
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroids is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
        
        return self._AbstractMesh__simplex_centroids
        
        
    
    def __compute_metrics(self): 
        raise NotImplementedError('This method must be implemented in the subclasses')       

In [3]:
vtx, faces, face_norm = utils.read_obj('bunny.obj')

In [4]:
prova = Trimesh()

In [5]:
prova.load_from_file('bunny.obj')

In [6]:
prova.simplex_centroids

array([[-0.44110933,  0.08367033,  0.44573333],
       [-0.133255  , -0.930293  ,  0.12635067],
       [-0.07569433, -0.96924433, -0.332939  ],
       ...,
       [-0.692062  ,  0.635649  , -0.563669  ],
       [-0.720955  ,  0.47139533,  0.563158  ],
       [-0.70555233, -0.29099833,  0.54905233]])

In [7]:
vtx[faces].mean(axis = 1)

array([[-0.44110933,  0.08367033,  0.44573333],
       [-0.133255  , -0.930293  ,  0.12635067],
       [-0.07569433, -0.96924433, -0.332939  ],
       ...,
       [-0.692062  ,  0.635649  , -0.563669  ],
       [-0.720955  ,  0.47139533,  0.563158  ],
       [-0.70555233, -0.29099833,  0.54905233]])

In [14]:
vtx[faces[0]]

array([[-0.435735,  0.070596,  0.449676],
       [-0.433542,  0.092247,  0.437848],
       [-0.454051,  0.088168,  0.449676]])

In [7]:
p = AbstractMesh()

In [19]:
faces[(faces[:,0] != 0) & (faces[:,1] != 0) & (faces[:,2] != 0)]

(28570, 3)

In [14]:
faces[faces[:,1] != 0].shape

(28575, 3)

In [16]:
faces[faces[:,2] != 0].shape

(28574, 3)

In [42]:
faces.shape

(28576, 3)

In [49]:
mappa = dict()
adjs =  [[] for i in range(28576)]

for idx, face in enumerate(faces):
    
    e1 = (face[1], face[0])
    e2 = (face[2], face[1])
    e3 = (face[0], face[2])
    
    try:
        if mappa[e1]:
            adjs[idx].append([mappa[e1]])
    except KeyError:
        mappa[(face[0], face[1])] = idx
    try:
        if mappa[e2]:
             adjs[idx].append([mappa[e1]])
    except KeyError:
        mappa[(face[1], face[2])] = idx
    try:
        if mappa[e3]:
             adjs[idx].append([mappa[e1]])
    except KeyError:
        mappa[(face[2], face[0])] = idx

In [50]:
adjs[0]

[]

In [28]:
adjs.spa

KeyboardInterrupt: 